In [1]:
import pandas as pd 
import numpy as np
import import_ipynb
from functions import *
from data_process import X_train, X_test, y_train, y_test

from sklearn import tree
from sklearn.impute import SimpleImputer

importing Jupyter notebook from functions.ipynb
importing Jupyter notebook from data_process.ipynb


In [ ]:
train = pd.read_csv("../data/raw/aps_failure_training_set.csv")
test = pd.read_csv("../data/raw/aps_failure_test_set.csv")

print('shape of training set is ',train.shape)
print('shape of test set is ',test.shape)

# Preprocessing data

In [ ]:
X_train = preprocess(train)
X_test = preprocess(test)

y_train = train['class']
y_test = test['class']

In [4]:
from data_process import X_train

importing Jupyter notebook from data_process.ipynb


In [5]:
X_train.head()

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,76698.0,0.713189,2.130706e+09,280.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,33058.0,0.713189,0.000000e+00,190620.639314,0.0,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,41040.0,0.713189,2.280000e+02,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,12.0,0.000000,7.000000e+01,66.000000,0.0,10.0,0.0,0.0,0.0,318.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,60874.0,0.713189,1.368000e+03,458.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


# The super fancy but copy-pastable machine learning

## Training/Fitting model using training data

clf = tree.DecisionTreeClassifier()
clf.fit(X_train,y_train)

## Using model to generate prediction using test data predictor variables

y_pred = clf.predict(X_test)
y_pred=pd.DataFrame(y_pred)

##Compute test dataset cost

cost(y_test,y_pred)

# JW's section

## SMOTE

In [ ]:
X_train_smote, y_train_smote = balance_data(X_train,y_train)

# Smote Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression(max_iter=99999)
clf.fit(X_train_smote,y_train_smote)
y_pred = clf.predict(X_test)
y_pred = pd.DataFrame(y_pred)
cost(y_test,y_pred)

# Smote Decision Tree

In [ ]:
## Testing
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_smote,y_train_smote)
y_pred = clf.predict(X_test)
y_pred = pd.DataFrame(y_pred)
cost(y_test,y_pred)

# LGBM

In [1]:
import lightgbm as lgb
from sklearn import metrics

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
clf = lgb.LGBMClassifier()
clf.fit(X_train_smote,y_train_smote)
y_pred = clf.predict(X_test)
y_pred = pd.DataFrame(y_pred)
cost(y_test,y_pred)

# Gradient Boosted Decision Tree

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier()
clf.fit(X_train_smote,y_train_smote)
y_pred = clf.predict(X_test)
y_pred = pd.DataFrame(y_pred)
cost(y_test,y_pred)

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train_smote,y_train_smote)
y_pred = clf.predict(X_test)
y_pred = pd.DataFrame(y_pred)
cost(y_test,y_pred)